## Python code for the illustrative example in Longstaff & Schwartz's paper

In [1]:
import pandas as pd
import numpy as np

In [2]:
disc=0.94176

In [3]:
np.exp(-0.06)

0.9417645335842487

In [4]:
price_path=pd.DataFrame({'Path':np.arange(1,9,1), 't=0':np.ones(8), 't=1':[1.09,1.16,1.22,.93,1.11,.76,.92,.88],
                      't=2':[1.08,1.26,1.07,0.97,1.56,.77,.84,1.22],
                      't=3':[1.34,1.54,1.03,.92,1.52,.90,1.01,1.34]})

In [5]:
price_path

,Path,t=0,t=1,t=2,t=3
0,1,1.0,1.09,1.08,1.34
1,2,1.0,1.16,1.26,1.54
2,3,1.0,1.22,1.07,1.03
3,4,1.0,0.93,0.97,0.92
4,5,1.0,1.11,1.56,1.52
5,6,1.0,0.76,0.77,0.90
6,7,1.0,0.92,0.84,1.01
7,8,1.0,0.88,1.22,1.34


In [6]:
CF=price_path.copy()

In [7]:
CF['t=0']=CF['t=1']=CF['t=2']=0

In [8]:
CF['t=3']=np.maximum(1.1-CF['t=3'],0.0)

In [9]:
CF

,Path,t=0,t=1,t=2,t=3
0,1,0,0,0,0.00
1,2,0,0,0,0.00
2,3,0,0,0,0.07
3,4,0,0,0,0.18
4,5,0,0,0,0.00
5,6,0,0,0,0.20
6,7,0,0,0,0.09
7,8,0,0,0,0.00


In [10]:
price_path['ITM_t1']=np.where(price_path['t=1']<1.1,True,False)
price_path['ITM_t2']=np.where(price_path['t=2']<1.1,True,False)  # ITM = in-the-money
price_path['ITM_t3']=np.where(price_path['t=3']<1.1,True,False)

In [11]:
price_path

,Path,t=0,t=1,t=2,t=3,ITM_t1,ITM_t2,ITM_t3
0,1,1.0,1.09,1.08,1.34,True,True,False
1,2,1.0,1.16,1.26,1.54,False,False,False
2,3,1.0,1.22,1.07,1.03,False,True,True
3,4,1.0,0.93,0.97,0.92,True,True,True
4,5,1.0,1.11,1.56,1.52,False,False,False
5,6,1.0,0.76,0.77,0.90,True,True,True
6,7,1.0,0.92,0.84,1.01,True,True,True
7,8,1.0,0.88,1.22,1.34,True,False,False


In [12]:
price_path.loc[price_path['ITM_t2']==True,'Y_reg_t2']=np.maximum(1.1-price_path.loc[price_path['ITM_t2'],'t=3'],0.0)*disc

In [13]:
price_path # Y_reg_t2， 用来做OLSmodel的Y值value，只选取ITM在t=2的时候

,Path,t=0,t=1,t=2,t=3,ITM_t1,ITM_t2,ITM_t3,Y_reg_t2
0,1,1.0,1.09,1.08,1.34,True,True,False,0.000000
1,2,1.0,1.16,1.26,1.54,False,False,False,NaN
2,3,1.0,1.22,1.07,1.03,False,True,True,0.065923
3,4,1.0,0.93,0.97,0.92,True,True,True,0.169517
4,5,1.0,1.11,1.56,1.52,False,False,False,NaN
5,6,1.0,0.76,0.77,0.90,True,True,True,0.188352
6,7,1.0,0.92,0.84,1.01,True,True,True,0.084758
7,8,1.0,0.88,1.22,1.34,True,False,False,NaN


In [14]:
N_basis=2
x=price_path.loc[price_path['ITM_t2'],'t=2'].values
y=price_path.loc[price_path['ITM_t2'],'Y_reg_t2'].values
reg = np.poly1d(np.polyfit(x,y,N_basis))  # Reminder: your will need this code for your final project!

这里就是用OLS去predict (discounted value of t=3 payoff to t=2) ，如果再t=2的时候不抛售put option，最后的获利可能是多少。 但这里限定了一个条件，在t=2时候，price要小于目标价格1.1（in the money)，且这里的获利是被削减过的，用discount pay off value。 可以理解成inflation rate

In [15]:
reg

poly1d([-1.81356745,  2.98339626, -1.0699825 ])

In [16]:
reg(1.5)

-0.6754148767124499

In [17]:
-1.07 + 2.983*3 - 1.813*(3**2)

-8.438

Regression of Y (discounted value of t=3 payoff to t=2) against X (t=2 asset price) and $X^2$ gives us
   $E(Y|X) = -1.070 + 2.983X - 1.813X^{2}$

In [18]:
reg(price_path['t=2'])

array([ 0.03674038, -0.1901229 ,  0.04589812,  0.11752626, -0.82938209,
        0.15196848,  0.15641716, -0.12955286])

In [19]:
price_path['Cont(t=2)']=np.maximum(reg(price_path['t=2']),0.0) # 用OLS predict的预期获利

In [20]:
price_path['Ex(t=2)']=np.maximum(1.1-price_path['t=2'],0) # 直接在t=2的时候卖出时候的获利

In [21]:
price_path

,Path,t=0,t=1,t=2,t=3,ITM_t1,ITM_t2,ITM_t3,Y_reg_t2,Cont(t=2),Ex(t=2)
0,1,1.0,1.09,1.08,1.34,True,True,False,0.000000,0.036740,0.02
1,2,1.0,1.16,1.26,1.54,False,False,False,NaN,0.000000,0.00
2,3,1.0,1.22,1.07,1.03,False,True,True,0.065923,0.045898,0.03
3,4,1.0,0.93,0.97,0.92,True,True,True,0.169517,0.117526,0.13
4,5,1.0,1.11,1.56,1.52,False,False,False,NaN,0.000000,0.00
5,6,1.0,0.76,0.77,0.90,True,True,True,0.188352,0.151968,0.33
6,7,1.0,0.92,0.84,1.01,True,True,True,0.084758,0.156417,0.26
7,8,1.0,0.88,1.22,1.34,True,False,False,NaN,0.000000,0.00


In [22]:
price_path['Ex(t=3)']=np.maximum(1.1-price_path['t=3'],0) # 真实在t=3才卖的获利情况
price_path

,Path,t=0,t=1,t=2,t=3,ITM_t1,ITM_t2,ITM_t3,Y_reg_t2,Cont(t=2),Ex(t=2),Ex(t=3)
0,1,1.0,1.09,1.08,1.34,True,True,False,0.000000,0.036740,0.02,0.00
1,2,1.0,1.16,1.26,1.54,False,False,False,NaN,0.000000,0.00,0.00
2,3,1.0,1.22,1.07,1.03,False,True,True,0.065923,0.045898,0.03,0.07
3,4,1.0,0.93,0.97,0.92,True,True,True,0.169517,0.117526,0.13,0.18
4,5,1.0,1.11,1.56,1.52,False,False,False,NaN,0.000000,0.00,0.00
5,6,1.0,0.76,0.77,0.90,True,True,True,0.188352,0.151968,0.33,0.20
6,7,1.0,0.92,0.84,1.01,True,True,True,0.084758,0.156417,0.26,0.09
7,8,1.0,0.88,1.22,1.34,True,False,False,NaN,0.000000,0.00,0.00


In [23]:
price_path['V(t=3)']=price_path['Ex(t=3)'] #?
price_path

,Path,t=0,t=1,t=2,t=3,ITM_t1,ITM_t2,ITM_t3,Y_reg_t2,Cont(t=2),Ex(t=2),Ex(t=3),V(t=3)
0,1,1.0,1.09,1.08,1.34,True,True,False,0.000000,0.036740,0.02,0.00,0.00
1,2,1.0,1.16,1.26,1.54,False,False,False,NaN,0.000000,0.00,0.00,0.00
2,3,1.0,1.22,1.07,1.03,False,True,True,0.065923,0.045898,0.03,0.07,0.07
3,4,1.0,0.93,0.97,0.92,True,True,True,0.169517,0.117526,0.13,0.18,0.18
4,5,1.0,1.11,1.56,1.52,False,False,False,NaN,0.000000,0.00,0.00,0.00
5,6,1.0,0.76,0.77,0.90,True,True,True,0.188352,0.151968,0.33,0.20,0.20
6,7,1.0,0.92,0.84,1.01,True,True,True,0.084758,0.156417,0.26,0.09,0.09
7,8,1.0,0.88,1.22,1.34,True,False,False,NaN,0.000000,0.00,0.00,0.00


In [24]:
price_path['V(t=2)']=np.maximum(price_path['Ex(t=2)'],price_path['Cont(t=2)']) #比较在t=2就卖的获利和预估如果在t=2保留，然后在t=3再卖的获利对比，选取最优策略
price_path

,Path,t=0,t=1,t=2,t=3,ITM_t1,ITM_t2,ITM_t3,Y_reg_t2,Cont(t=2),Ex(t=2),Ex(t=3),V(t=3),V(t=2)
0,1,1.0,1.09,1.08,1.34,True,True,False,0.000000,0.036740,0.02,0.00,0.00,0.036740
1,2,1.0,1.16,1.26,1.54,False,False,False,NaN,0.000000,0.00,0.00,0.00,0.000000
2,3,1.0,1.22,1.07,1.03,False,True,True,0.065923,0.045898,0.03,0.07,0.07,0.045898
3,4,1.0,0.93,0.97,0.92,True,True,True,0.169517,0.117526,0.13,0.18,0.18,0.130000
4,5,1.0,1.11,1.56,1.52,False,False,False,NaN,0.000000,0.00,0.00,0.00,0.000000
5,6,1.0,0.76,0.77,0.90,True,True,True,0.188352,0.151968,0.33,0.20,0.20,0.330000
6,7,1.0,0.92,0.84,1.01,True,True,True,0.084758,0.156417,0.26,0.09,0.09,0.260000
7,8,1.0,0.88,1.22,1.34,True,False,False,NaN,0.000000,0.00,0.00,0.00,0.000000


In [25]:
# 选取t=2的最佳策略，如果预测的要比立马卖低，选择立马卖
price_path['CF_t=2']=0
price_path.loc[(price_path.ITM_t2) & (price_path['Cont(t=2)']<price_path['Ex(t=2)']),'CF_t=2']=price_path['Ex(t=2)']
price_path

,Path,t=0,t=1,t=2,t=3,ITM_t1,ITM_t2,ITM_t3,Y_reg_t2,Cont(t=2),Ex(t=2),Ex(t=3),V(t=3),V(t=2),CF_t=2
0,1,1.0,1.09,1.08,1.34,True,True,False,0.000000,0.036740,0.02,0.00,0.00,0.036740,0.00
1,2,1.0,1.16,1.26,1.54,False,False,False,NaN,0.000000,0.00,0.00,0.00,0.000000,0.00
2,3,1.0,1.22,1.07,1.03,False,True,True,0.065923,0.045898,0.03,0.07,0.07,0.045898,0.00
3,4,1.0,0.93,0.97,0.92,True,True,True,0.169517,0.117526,0.13,0.18,0.18,0.130000,0.13
4,5,1.0,1.11,1.56,1.52,False,False,False,NaN,0.000000,0.00,0.00,0.00,0.000000,0.00
5,6,1.0,0.76,0.77,0.90,True,True,True,0.188352,0.151968,0.33,0.20,0.20,0.330000,0.33
6,7,1.0,0.92,0.84,1.01,True,True,True,0.084758,0.156417,0.26,0.09,0.09,0.260000,0.26
7,8,1.0,0.88,1.22,1.34,True,False,False,NaN,0.000000,0.00,0.00,0.00,0.000000,0.00


In [26]:
# 加上如果不在t=2卖，在t=3的时候，获利情况
price_path['CF_t=3']=0
price_path.loc[price_path['CF_t=2']==0.0, 'CF_t=3']=np.maximum(0.0, price_path.loc[price_path['CF_t=2']==0.0, 'Ex(t=3)'])
price_path

,Path,t=0,t=1,t=2,t=3,ITM_t1,ITM_t2,ITM_t3,Y_reg_t2,Cont(t=2),Ex(t=2),Ex(t=3),V(t=3),V(t=2),CF_t=2,CF_t=3
0,1,1.0,1.09,1.08,1.34,True,True,False,0.000000,0.036740,0.02,0.00,0.00,0.036740,0.00,0.00
1,2,1.0,1.16,1.26,1.54,False,False,False,NaN,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00
2,3,1.0,1.22,1.07,1.03,False,True,True,0.065923,0.045898,0.03,0.07,0.07,0.045898,0.00,0.07
3,4,1.0,0.93,0.97,0.92,True,True,True,0.169517,0.117526,0.13,0.18,0.18,0.130000,0.13,0.00
4,5,1.0,1.11,1.56,1.52,False,False,False,NaN,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00
5,6,1.0,0.76,0.77,0.90,True,True,True,0.188352,0.151968,0.33,0.20,0.20,0.330000,0.33,0.00
6,7,1.0,0.92,0.84,1.01,True,True,True,0.084758,0.156417,0.26,0.09,0.09,0.260000,0.26,0.00
7,8,1.0,0.88,1.22,1.34,True,False,False,NaN,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00


In [27]:
# 重复在t=2时候的OLs，继续对t=1 respect t=2时候。因为只能exercise一次，所以对应的是t=2的最佳策略
price_path.loc[price_path['ITM_t1'],'Y_reg_t1']=price_path.loc[price_path['ITM_t1'],'CF_t=2']*disc
price_path

,Path,t=0,t=1,t=2,t=3,ITM_t1,ITM_t2,ITM_t3,Y_reg_t2,Cont(t=2),Ex(t=2),Ex(t=3),V(t=3),V(t=2),CF_t=2,CF_t=3,Y_reg_t1
0,1,1.0,1.09,1.08,1.34,True,True,False,0.000000,0.036740,0.02,0.00,0.00,0.036740,0.00,0.00,0.000000
1,2,1.0,1.16,1.26,1.54,False,False,False,NaN,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,NaN
2,3,1.0,1.22,1.07,1.03,False,True,True,0.065923,0.045898,0.03,0.07,0.07,0.045898,0.00,0.07,NaN
3,4,1.0,0.93,0.97,0.92,True,True,True,0.169517,0.117526,0.13,0.18,0.18,0.130000,0.13,0.00,0.122429
4,5,1.0,1.11,1.56,1.52,False,False,False,NaN,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,NaN
5,6,1.0,0.76,0.77,0.90,True,True,True,0.188352,0.151968,0.33,0.20,0.20,0.330000,0.33,0.00,0.310781
6,7,1.0,0.92,0.84,1.01,True,True,True,0.084758,0.156417,0.26,0.09,0.09,0.260000,0.26,0.00,0.244858
7,8,1.0,0.88,1.22,1.34,True,False,False,NaN,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000


In [28]:
N_basis=2
x=price_path.loc[price_path['ITM_t1'],'t=1'].values
y=price_path.loc[price_path['ITM_t1'],'Y_reg_t1'].values
reg = np.poly1d(np.polyfit(x,y,N_basis))

In [29]:
reg

poly1d([ 1.35645006, -3.33542735,  2.03750253])

Regression of Y (discounted value of t=2 payoff to t=1) against X (t=1 asset price) and $X^2$ gives us
   $E(Y|X) = 2.038 -3.335X + 1.356X^{2}$

In [30]:
price_path['Cont(t=1)']=np.maximum(reg(price_path['t=1']),0.0) # 用OLS predict的预期获利

In [31]:
price_path['Ex(t=1)']=np.maximum(1.1-price_path['t=1'],0) # 直接在t=1的时候卖出时候的获利

In [32]:
price_path['V(t=1)']=np.maximum(price_path['Ex(t=1)'],price_path['Cont(t=1)']) #比较predict和事实哪个大，就选哪个

In [33]:
price_path

,Path,t=0,t=1,t=2,t=3,ITM_t1,ITM_t2,ITM_t3,Y_reg_t2,Cont(t=2),Ex(t=2),Ex(t=3),V(t=3),V(t=2),CF_t=2,CF_t=3,Y_reg_t1,Cont(t=1),Ex(t=1),V(t=1)
0,1,1.0,1.09,1.08,1.34,True,True,False,0.000000,0.036740,0.02,0.00,0.00,0.036740,0.00,0.00,0.000000,0.013485,0.01,0.013485
1,2,1.0,1.16,1.26,1.54,False,False,False,NaN,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,NaN,0.000000,0.00,0.000000
2,3,1.0,1.22,1.07,1.03,False,True,True,0.065923,0.045898,0.03,0.07,0.07,0.045898,0.00,0.07,NaN,0.000000,0.00,0.000000
3,4,1.0,0.93,0.97,0.92,True,True,True,0.169517,0.117526,0.13,0.18,0.18,0.130000,0.13,0.00,0.122429,0.108749,0.17,0.170000
4,5,1.0,1.11,1.56,1.52,False,False,False,NaN,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,NaN,0.006460,0.00,0.006460
5,6,1.0,0.76,0.77,0.90,True,True,True,0.188352,0.151968,0.33,0.20,0.20,0.330000,0.33,0.00,0.310781,0.286063,0.34,0.340000
6,7,1.0,0.92,0.84,1.01,True,True,True,0.084758,0.156417,0.26,0.09,0.09,0.260000,0.26,0.00,0.244858,0.117009,0.18,0.180000
7,8,1.0,0.88,1.22,1.34,True,False,False,NaN,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.152761,0.22,0.220000


In [34]:
price_path=price_path.set_index('Path')
price_path

,t=0,t=1,t=2,t=3,ITM_t1,ITM_t2,ITM_t3,Y_reg_t2,Cont(t=2),Ex(t=2),Ex(t=3),V(t=3),V(t=2),CF_t=2,CF_t=3,Y_reg_t1,Cont(t=1),Ex(t=1),V(t=1)
Path,,,,,,,,,,,,,,,,,,,
1,1.0,1.09,1.08,1.34,True,True,False,0.000000,0.036740,0.02,0.00,0.00,0.036740,0.00,0.00,0.000000,0.013485,0.01,0.013485
2,1.0,1.16,1.26,1.54,False,False,False,NaN,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,NaN,0.000000,0.00,0.000000
3,1.0,1.22,1.07,1.03,False,True,True,0.065923,0.045898,0.03,0.07,0.07,0.045898,0.00,0.07,NaN,0.000000,0.00,0.000000
4,1.0,0.93,0.97,0.92,True,True,True,0.169517,0.117526,0.13,0.18,0.18,0.130000,0.13,0.00,0.122429,0.108749,0.17,0.170000
5,1.0,1.11,1.56,1.52,False,False,False,NaN,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,NaN,0.006460,0.00,0.006460
6,1.0,0.76,0.77,0.90,True,True,True,0.188352,0.151968,0.33,0.20,0.20,0.330000,0.33,0.00,0.310781,0.286063,0.34,0.340000
7,1.0,0.92,0.84,1.01,True,True,True,0.084758,0.156417,0.26,0.09,0.09,0.260000,0.26,0.00,0.244858,0.117009,0.18,0.180000
8,1.0,0.88,1.22,1.34,True,False,False,NaN,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.152761,0.22,0.220000


In [35]:
# You can also calculate the option price this way

In [36]:
price_path['V(t=1)'].mean()*disc

0.10947316499680305

In [37]:
stopping_rule=pd.DataFrame()
stopping_rule['Path']=price_path.index
stopping_rule.set_index('Path',inplace=True)

In [38]:
stopping_rule

""
Path
1
2
3
4
5
6
7
8


In [39]:
stopping_rule['t=1']=np.where(price_path['Cont(t=1)']<price_path['Ex(t=1)'],1,0)

In [40]:
stopping_rule['t=2']=np.where((price_path['Cont(t=2)']<price_path['Ex(t=2)']) & ~(stopping_rule['t=1'])
                              ,1,0)

In [41]:
stopping_rule['t=3']=np.where((price_path['ITM_t3']) & ~(stopping_rule['t=1']) & ~(stopping_rule['t=2'])
                              ,1,0)

In [42]:
stopping_rule

,t=1,t=2,t=3
Path,,,
1,0,0,0
2,0,0,0
3,0,0,1
4,1,0,0
5,0,0,0
6,1,0,0
7,1,0,0
8,1,0,0


In [43]:
CF[['t={}'.format(i) for i in [1,2,3]]]=(1.1-price_path[['t={}'.format(i) for i in [1,2,3]]]).values * stopping_rule.values

In [44]:
CF

,Path,t=0,t=1,t=2,t=3
0,1,0,0.00,0.0,-0.00
1,2,0,-0.00,-0.0,-0.00
2,3,0,-0.00,0.0,0.07
3,4,0,0.17,0.0,0.00
4,5,0,-0.00,-0.0,-0.00
5,6,0,0.34,0.0,0.00
6,7,0,0.18,0.0,0.00
7,8,0,0.22,-0.0,-0.00


In [45]:
for i in [1,2,3]:
    CF['t={}_disc'.format(i)]=CF['t={}'.format(i)]*(disc**i)

In [46]:
0.07*(disc**3)

0.058468070407864337

In [47]:
CF


,Path,t=0,t=1,t=2,t=3,t=1_disc,t=2_disc,t=3_disc
0,1,0,0.00,0.0,-0.00,0.000000,0.0,-0.000000
1,2,0,-0.00,-0.0,-0.00,-0.000000,-0.0,-0.000000
2,3,0,-0.00,0.0,0.07,-0.000000,0.0,0.058468
3,4,0,0.17,0.0,0.00,0.160099,0.0,0.000000
4,5,0,-0.00,-0.0,-0.00,-0.000000,-0.0,-0.000000
5,6,0,0.34,0.0,0.00,0.320198,0.0,0.000000
6,7,0,0.18,0.0,0.00,0.169517,0.0,0.000000
7,8,0,0.22,-0.0,-0.00,0.207187,-0.0,-0.000000


### Calcuate American Option Price by averging discounted payoff across all paths

In [48]:
opt_am=CF[['t={}_disc'.format(i) for i in [1,2,3]]].sum(axis=1).mean()

In [49]:
opt_am

0.11443370880098308

In [50]:
CF.iloc[:,5:8].sum(axis=1)

0    0.000000
1    0.000000
2    0.058468
3    0.160099
4    0.000000
5    0.320198
6    0.169517
7    0.207187
dtype: float64

### Calculate European Option Price as a comparison

In [51]:
opt_eu=price_path['Ex(t=3)'].mean()*(disc**3)

In [52]:
opt_eu

0.05637992503615492